# Problema de optimización usando Algoritmos Genéticos
<a href="https://colab.research.google.com/github/milocortes/mod_04_concentracion/blob/main/src/talleres_clase/prob_opt_algo_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Función Himmelblau

* Función: $f(x,y) = (x^2 +y -11)^2 + (x+y^2-7)^2$
* NO Var: 2
* Precisión: 6
* Espacio de búsqueda : $-5<x,y < 5$, $-∞ < z < 1000$

# Parámetros del algoritmo

* Método de selección: ruleta
* Cruza: En un punto
* Mutación: en un bit
* Número de generación: 30
* Tamaño de la población (número de individuos): 50
* Probabilidad de cruza: $P_c = 0.85$
* Probabilidad de mutación : 0.000000001

## Generación de la población

### Codificación de las variables

$$
L = [\log_2((l_{sup} - l_{inf})\times 10^{\text{precision}})]
$$


In [1]:
from math import log2, ceil


def length_variable(l_sup: int, l_inf: int , precision: int):
    """
    Nombre de la función : length_variable
    
    Descripción: Función que obtiene la cantidad de bits para representar la variable
    
    argumentos:
        * l_sup: límite superior de la función
    """
    return ceil(log2((l_sup - l_inf)*10**precision))

x_l_sup = 5
y_l_sup = 5

x_l_inf = -5
y_l_inf = -5

precision = 6

L_x = length_variable(x_l_sup, x_l_inf, precision)
L_y = length_variable(y_l_sup, y_l_inf, precision)
L_x, L_y

(24, 24)

In [4]:
# Función que genera poblacipob aleatori de m individuos de tamaño L
import random

m = 30 

def rand_population_binary(m,L):
    return [[random.randint(0,1)  for i in range(L)] for j in range(m)]

rand_population_binary(m,L_x)[0]

[0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1]

In [5]:
rand_population_binary(m,L_y)[0]

[0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0]

In [45]:
# GENERAMOS LA POBLACIÓN INICIAL

import numpy as np

n_variables = 2
l_sup_vec = [x_l_sup, y_l_sup]
l_inf_vec = [x_l_inf, y_l_inf]

dimension_vec = []
genotipo = []
L_cromosoma = 0

# Generamos la poblacion inicial

for i in range(n_variables):
    L_var = length_variable(l_sup_vec[i], l_inf_vec[i], precision)
    L_cromosoma +=  L_var
    
    dimension_vec.append(L_var)
    genotipo.append(rand_population_binary(m, L_var))

genotipo

[[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1],
  [0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1],
  [1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1],
  [0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1],
  [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0],
  [1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0],
  [0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
  [0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1],
  [1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0],
  [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1],
  [0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0],
  [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1],
  [1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0],
  [0, 0, 0, 

# Evaluación de población inicial

In [9]:
# Función que obtiene las potencias base 2 de un vector de bits (un individui)

def to_decimal(dimension,individuo):
    return sum([2**(i) for i in range(dimension-1,-1,-1) ]* np.array(individuo))

In [10]:
dimension = 24
individuo_inicial = genotipo[0][4]

to_decimal(dimension, individuo_inicial)

4151461

In [23]:
## la variable j representa cada bit de un individuo (secuencia de bits)
[(2**i)*j for i,j in zip(range(4-1,-1,-1),[1,0,1,0])]

[8, 0, 2, 0]

In [25]:
[2**i for i in range(4-1, -1,-1)] * np.array([1,0,1,0])

array([8, 0, 2, 0])

In [26]:
# Función que decodifica el vector a un valor real

def binary2real(i_sup, i_inf, dimension, individuo):
    return i_inf+ (to_decimal(dimension, individuo)* ((i_sup-i_inf)/(2**len(individuo)-1)))

In [29]:
for i in genotipo[1]:
    print(binary2real(5, -5, 24, i))

4.498438805248666
-0.26912541801484924
0.46891662293175607
-2.2116915709788545
2.4426107074386305
-3.075847511043996
-1.1840996851980496
4.212160659561196
0.07014006794333838
-0.10897309237558161
0.05360037407877272
-2.913392061793331
-3.840396931195076
-3.0647354164561875
4.279038267078297
-1.9988868831924727
-3.2620566047463777
0.5173805664408544
-1.291990655183235
2.2969959555265875
1.6975412784541417
1.9990555643472412
0.8161905894393078
-0.22832186390887887
0.8550122889883687
-2.1406511748225197
4.2033946039315815
-4.2008095503335925
2.025190414499665
-3.044195654642323


In [33]:
def binary2real(i_sup, i_inf, dimension, poblacion):
    return [i_inf+ (to_decimal(dimension, individuo)* ((i_sup-i_inf)/(2**len(individuo)-1))) for individuo in poblacion]

In [36]:
binary2real(5, -5, 24,genotipo[0])

[-4.669150094339257,
 -2.0055351856669894,
 -0.8143666872004678,
 -2.276953296479779,
 -2.525536270471589,
 -1.535677107314891,
 2.10029286743956,
 -0.6203595173573202,
 -2.8455458787408996,
 -4.130929060633722,
 -1.6860560587677988,
 -2.983713625890829,
 -3.4686790984081686,
 4.572345588943099,
 -2.199786734568282,
 -3.104051238539889,
 -1.5881041638913254,
 3.3222990228115936,
 4.289362388215208,
 -0.6021562577579171,
 0.44094475751786,
 -1.5657869914643165,
 3.6525409610593886,
 4.9383419715369925,
 -1.520252616420544,
 0.9962335822721471,
 2.3666362384936948,
 -2.3749934062357787,
 0.17493636458732897,
 -2.4605970061181193]

In [57]:
# Generamos la lista que contiene el fenotipo, es decir, realizamos la decodificación
## Bits ---> reales

feno = [[] for i in range(m)]

for i in range(n_variables):
  # Obtenemos los limites para cada variable
  i_sup = l_sup_vec[i]
  i_inf = l_inf_vec[i]

  pob = genotipo[i]

  dim = dimension_vec[i]

  b2r = binary2real(i_sup, i_inf, dim, pob)
  
  for j in range(m):
    feno[j].append(b2r[j])

In [59]:
feno

[[4.991971253870204, 2.9455225435210792],
 [-2.2526852639129915, 2.727773054109398],
 [2.2934542473229316, -2.131630011298061],
 [-1.7015973747728692, 2.999552965137539],
 [0.17830521931083343, -4.445341792425024],
 [0.27970226286067135, 2.383500181645166],
 [-1.1033568443868664, -2.9476379124902437],
 [-0.2932277496592848, -4.740863426975216],
 [4.806613314545949, -2.6273243205144596],
 [-4.444188442479875, -3.3155690619688665],
 [-4.343117436356391, -4.154809066939895],
 [0.17775387631379846, 1.3613561607215496],
 [2.5848273983494874, -2.2376040957930146],
 [-4.847907414907659, 4.774323688407165],
 [-0.7157233784033883, -1.4734617753900157],
 [-0.7762173280845479, -1.0563418898786239],
 [2.4853347233137324, 0.616764164970169],
 [-0.827195991706609, 0.6983396827184967],
 [-1.437112476653604, -1.5664164165506613],
 [2.5686673860947717, -3.8199698221665512],
 [4.440132346161148, 1.3321707446676934],
 [4.687185268830374, 1.6698030632616918],
 [2.059545937749501, 0.6601754224404948],
 [-4

In [60]:
def f_himmelblau(X):
  x,y = X
  return (x**2 + y -11)**2 + (x + y**2 -7)**2

In [68]:
objv = [f_himmelblau(f) for f in feno]
objv

[328.9015436551873,
 13.508000788210824,
 61.99009457191005,
 26.14861258902506,
 405.0047608196211,
 73.98198199813015,
 162.40152442407097,
 475.58537245947304,
 111.97738895349343,
 29.74542660196467,
 48.78658623863281,
 116.98587875431804,
 43.33480317120448,
 418.2990804973696,
 173.81331270002426,
 175.55057231117001,
 34.7855219546027,
 146.36303925626376,
 146.07530720245325,
 170.8425354184748,
 101.55764523866166,
 159.9832142422052,
 57.47838757646295,
 178.71584929149273,
 134.3504332792021,
 159.61864610294958,
 77.97880904313226,
 510.4825570842903,
 47.02356565690665,
 49.99985106741518]

## Selección

En la selección vamos a escoger a los individuos con las mejores condiciones para contribuir a la generación de la siguiente generación.

Para esto, tenemos que calcular la aptitud de cada individuo.

### Aptitud

In [73]:
## Generación de la aptitud con normalización
## Como estamos en un problema de minimización, para obtener la aptitud de los valores de la función objetivo,
## la propuesta es usar la inversa de estos valores, pues la inversa de un valor muy pequeño, es superior a la inversa de un
## valor grande, de manera que en el primer caso, la aptitud será mayor
val_min = min(objv)
val_max = max(objv)

objv_norm = [ (((i-val_min)/(val_max-val_min))+0.0001)**-1 for i in objv]

for i,j in zip(objv, objv_norm):
  print(f"{i}--{j}")

328.9015436551873--1.5754801269091072
13.508000788210824--10000.0
61.99009457191005--10.240185734033753
26.14861258902506--39.16173721642972
405.0047608196211--1.2692607653397792
73.98198199813015--8.211241736351454
162.40152442407097--3.336671200301277
475.58537245947304--1.0754067214349203
111.97738895349343--5.044449541567698
29.74542660196467--30.513341618178366
48.78658623863281--14.067328367407002
116.98587875431804--4.80040756553576
43.33480317120448--16.634296486551904
418.2990804973696--1.2275802842206989
173.81331270002426--3.0992144134225
175.55057231117001--3.065997900009199
34.7855219546027--23.302361414485002
146.36303925626376--3.73932917320159
146.07530720245325--3.7474421966562192
170.8425354184748--3.157715037328848
101.55764523866166--5.6410700187282465
159.9832142422052--3.391741057618602
57.47838757646295--11.28972297151278
178.71584929149273--3.007273001740228
134.3504332792021--4.110892549544088
159.61864610294958--3.4002010865346235
77.97880904313226--7.70258309

In [80]:
## Generacion de aptitud escalada

def scaled_fitness(new_max, new_min, original_fitness):
  val_max = max(original_fitness)
  val_min = min(original_fitness)

  y = np.array([new_min, new_max])
  X = np.matrix([[val_min, 1],[1, val_max]])

  a,b = np.ravel(X.I @ y)

  return [()]
scaled_fitness(100,50, objv_norm)

(49.99999949994999, 0.005000000050005002)